# Reed Solomon Code
https://pypi.org/project/reedsolo/#table-of-contents 

Reed-Solomon codes are the subclass of non-binary BCH codes introduced by Irving S.Reed and Gustacvae Solomon.
RS code can operates on multiple bits rather than individual bits. RS codes help in recovering corrupted messages that are being transferred over a network with encoder and decoder.

Encoder receives data, and before transferring it over the noisy network it adds some parity bits with origianl data bits. Decoder detects corrupted messages and recovers them from error.

## Parameters:
1. (n,k) code can encode m-bit symbols.
2. BLock length n is given by 2m-1 symbols.
3. Message size is given by (n-2t) where t is the number of errors corrected.
4. Parity check size is given by (n-k) or 2t symbols.
5. Minimum distance d is given by (2t+1).
6. Message size is of k bits.

## Generator Function
The generator function is generated using a special polynomial. All valid codewords are exactly divisible by the generator polynomial given as follow:
$$ g(x) = (x-\alpha)(x-\alpha^2)(x-\alpha^3)(x-\alpha^4)\dots (x-\alpha^{2t})$$

## Encoding
A RS code with n block size, k message size, q symbols in bits can encode the message as a polynomial p(x) and then multiply it with a code generator polynomial g(x).

We then map the mssage vector $[x_1, x_2, ...., x_k]$ to a polynomial p(x) of degree < k s.t. 
$$p_x(\alpha_i) = x_i$$ for all i = 1,2,3,...., k
Polynomial can be done using Lagrange interpolation. 
Sender calculates 
$$s(x) = p(x) \times g(x)$$
and then sends over the coefficients of $s(x)$

## Decoding
The receiver receives r(x) at the receiver end. 
If $s(x) == r(x)$, then \frac{r(x)}{g(x)} has no remainder.
If it has remainder, then $r(x) = p(x) \times g(x) + e(x)$, where e(x) is an error polynomial.

It can protect your data from errors and bitrot. This burst-type implementation supports any Galois field higher than $2^3$, but not binary streams. Burst errors are non-random errors that more often happen on data storage mediums such as hard drive, hense this library is better suited for data storage protection, and less for streams noise correction. Although it work for the purpose but with a bit overhead (it works on bytes, but not bits.)

In [1]:
pip install --upgrade reedsolo


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Initialization

In [2]:
from reedsolo import RSCodec, ReedSolomonError

In [3]:
rsc = RSCodec(10) # 10 error correction code (ecc) symbols 

# Encoding

In [4]:
# just a list of numbers/symbols:
rsc.encode([1,2,3,4])

bytearray(b'\x01\x02\x03\x04,\x9d\x1c+=\xf8h\xfa\x98M')

In [5]:
# bytearrays are accepted and the output will be matched:
rsc.encode(bytearray([1,2,3,4]))

bytearray(b'\x01\x02\x03\x04,\x9d\x1c+=\xf8h\xfa\x98M')

In [6]:
# encoding a byte string is as easy:
rsc.encode(b'hello world')
# Note: strings of any length, even if longer than the Galois field, will be encoded as well using transparent chunking.

bytearray(b'hello world\xed%T\xc4\xfd\xfd\x89\xf3\xa8\xaa')

# Decoding (repairing)

## String

In [7]:
rsc.decode(b'hello world\xed%T\xc4\xfd\xfd\x89\xf3\xa8\xaa')[0]  # original

bytearray(b'hello world')

In [8]:
rsc.decode(b'heXlo worXd\xed%T\xc4\xfdX\x89\xf3\xa8\xaa')[0]     # 3 errors out of 10 alephabts: hello world --> heXlo worXd

bytearray(b'hello world')

In [9]:
rsc.decode(b'hXXXo worXd\xed%T\xc4\xfdX\x89\xf3\xa8\xaa')[0]     # 5 errors: hello world --> hXXXO worXd 

bytearray(b'hello world')

In [10]:
# rsc.decode(b'hXXXo worXd\xed%T\xc4\xfdXX\xf3\xa8\xaa')[0]        # 6 errors - fail hello world --> hXXXo worXd

In [44]:
rsc.encode(b'How are you?')

bytearray(b'How are you?X~\xa4[\xce\x14@\x91Q\xc7\xd2dJ\xadR\xb7?\x85\x85\xb5')

In [46]:
rsc.decode(b'How are you?X~\xa4[\xce\x14@\x91Q\xc7\xd2dJ\xadR\xb7?\x85\x85\xb5')[0]  # original

bytearray(b'How are you?')

In [49]:
rsc.decode(b'HXw axe yYu?X~\xa4[\xce\x14@\x91Q\xc7\xd2dJ\xadR\xb7?\x85\x85\xb5')[0] # 3 errors out of 9 alphabets

bytearray(b'How are you?')

## List

In [52]:
rsc.encode(b'[1,2,4,7,12]')

bytearray(b'[1,2,4,7,12]h\xf4\x1e\xc9q\xc8\xa4\xe4\x7f\xd9\x00\x12u\xb5\x82TLnfQ')

In [55]:
rsc.decode(b'[1,2,4,7,12]h\xf4\x1e\xc9q\xc8\xa4\xe4\x7f\xd9\x00\x12u\xb5\x82TLnfQ')[0]

bytearray(b'[1,2,4,7,12]')

In [57]:
rsc.decode(b'[1,2,5,8,12]h\xf4\x1e\xc9q\xc8\xa4\xe4\x7f\xd9\x00\x12u\xb5\x82TLnfQ')[0] 
# 1 error corrected 7->8

bytearray(b'[1,2,4,7,12]')

In [60]:
rsc.decode(b'[1,2,5,8,34]h\xf4\x1e\xc9q\xc8\xa4\xe4\x7f\xd9\x00\x12u\xb5\x82TLnfQ')[0] 
# 2 error corrected 12->34

bytearray(b'[1,2,4,7,12]')

In [64]:
rsc.decode(b'[1,2,5,8134]h\xf4\x1e\xc9q\xc8\xa4\xe4\x7f\xd9\x00\x12u\xb5\x82TLnfQ')[0] 
# 4 error corrected 7->8, ,->1, 12->34

bytearray(b'[1,2,4,7,12]')

# Number and Letter

In [14]:
rsc.encode(b'[HH0H000H000000V00000000000]') #1,2,4,7, 12

bytearray(b'[HH0H000H000000V00000000000]I\xa1&\xd2C\nV`\xc7\xc4')

In [15]:
rsc.decode(b'[HH0H000H000000V00000000000]I\xa1&\xd2C\nV`\xc7\xc4')[0]  
# original

bytearray(b'[HH0H000H000000V00000000000]')

In [65]:
# rsc.decode(b'[H00H000H000000V00000000000]Shx\x016\xa5\xa8\xf6"\xb5')[0]  
# original #1,2,4,7 --> 3,4,7,12 it will give an error
# becasue this code can only detect the difference but cannot detecf the loss or add error

# RSCodec.decode() returns 3 variables:
1. the decoded (corrected) message
2. the decoded message and error correction code (which is itself also correctd)
3. and the list of position of the errata (errors and erasures)


In [23]:
tampered_msg = b'heXlo worXd\xed%T\xc4\xfdX\x89\xf3\xa8\xaa' # from the reference ecc symbols \xed%T\xc4\xfdX\x89\xf3\xa8\xaa
decoded_msg, decoded_msgecc, errata_pos = rsc.decode(tampered_msg)

### decoded/corrected message

In [24]:
print(decoded_msg) 

bytearray(b'hello world')


### decoded/corrected message and ecc symbols

In [25]:
print(decoded_msgecc) 

bytearray(b'hello world\xed%T\xc4\xfd\xfd\x89\xf3\xa8\xaa')


### the list of position of the errata (errors and erasures)

In [26]:
print(errata_pos)  # errata_pos is returned as a bytearray, hardly intelligible

bytearray(b'\x10\t\x02')


In [27]:
print(list(errata_pos))  # convert to a list to get the errata positions as integer indices
# errors happen at the 9th and 2nd position

[16, 9, 2]


Since we failed to decode with 6 errors with a codec set with 10 error correction code (ecc) symbols, let’s try to use a bigger codec, with 12 ecc symbols.

In [75]:
rsc = RSCodec(12)  # using 2 more ecc symbols (to correct max 6 errors or 12 erasures)

In [76]:
rsc.encode(b'hello world')

bytearray(b'hello world?Ay\xb2\xbc\xdc\x01q\xb9\xe3\xe2=')

In [77]:
rsc.decode(b'hello worXXXXy\xb2XX\x01q\xb9\xe3\xe2=')[0]

bytearray(b'hello world')

In [79]:
rsc.decode(b'helXXXXXXXXXXy\xb2XX\x01q\xb9\xe3\xe2=', erase_pos=[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 16])[0]  
# 12 erasures - OK

bytearray(b'hello world')

This shows that we can decode twice as many erasures (where we provide the location of errors ourselves) than errors (with unknown locations). This is the cost of error correction compared to erasure correction. 

\

To get the maximum number of errors or erasures that can be independently corrected (ie, not simultaneously):


In [80]:
maxerrors, maxerasures = rsc.maxerrata(verbose=True)

This codec can correct up to 6 errors and 12 erasures independently


In [81]:
# print(maxerrors, maxerasures)

To get the maximum number of errors and erasures that can be simultaneously corrected, you need to specify the number of errors or erasures you expect:

In [82]:
maxerrors, maxerasures = rsc.maxerrata(erasures=6, verbose=True)  # we know the number of erasures, will calculate how many errors we can afford

This codec can correct up to 3 errors and 6 erasures simultaneously


In [83]:
print(maxerrors, maxerasures)

3 6


In [84]:
maxerrors, maxerasures = rsc.maxerrata(errors=5, verbose=True)  # we know the number of errors, will calculate how many erasures we can afford

This codec can correct up to 5 errors and 2 erasures simultaneously


In [85]:
print(maxerrors, maxerasures)

5 2


Note that if a chunk has more errors and erasures than the Singleton Bound as calculated by the maxerrata() method, the codec will try to raise a `ReedSolomonError`exception, but may very well not detect any error either (this is a theoretical limitation of error correction codes). In other words, error correction codes are unreliable to detect if a chunk of a message is corrupted beyond the Singleton Bound. If you want more reliability in errata detection, use a checksum or hash such as SHA or MD5 on your message, these are much more reliable and have no bounds on the number of errata (the only potential issue is with collision but the probability is very very low).

In [86]:
from reedsolo import ReedSolomonError

To check if a message is tampered given its error correction symbols, without decoding, use the check() method:

In [41]:
rsc.check(b'hello worXXXXy\xb2XX\x01q\xb9\xe3\xe2=')

[False]

In [88]:
rmes, rmesecc, errata_pos = rsc.decode(b'hello worXXXXy\xb2XX\x01q\xb9\xe3\xe2=')

In [89]:
rsc.check(rmesecc)  # Corrected or untampered message will return True

[True]

In [90]:
print('Number of detected errors and erasures: %i, their positions: %s' % (len(errata_pos), list(errata_pos)))

Number of detected errors and erasures: 6, their positions: [16, 15, 12, 11, 10, 9]


By default, most Reed-Solomon codecs are limited to characters that can be encoded in 256 bits and with a length of maximum 256 characters. But this codec is universal, you can reduce or increase the length and maximum character value by increasing the Galois Field:

In [91]:
# To use longer chunks or bigger values than 255 (may be very slow)
rsc = RSCodec(12, nsize=4095)  # always use a power of 2 minus 1
rsc = RSCodec(12, c_exp=12)  # alternative way to set nsize=4095
mes = 'a' * (4095-12)
mesecc = rsc.encode(mes)
mesecc[2] = 1
mesecc[-1] = 1
rmes, rmesecc, errata_pos = rsc.decode(mesecc)
rsc.check(mesecc)

[False]

In [92]:
rsc.check(rmesecc)

[True]

# Low-level usage via direct access to math functions

In [93]:
import reedsolo as rs

In [94]:
rs.init_tables(0x11d)

[bytearray(b'\x00\x00\x01\x19\x022\x1a\xc6\x03\xdf3\xee\x1bh\xc7K\x04d\xe0\x0e4\x8d\xef\x81\x1c\xc1i\xf8\xc8\x08Lq\x05\x8ae/\xe1$\x0f!5\x93\x8e\xda\xf0\x12\x82E\x1d\xb5\xc2}j\'\xf9\xb9\xc9\x9a\txM\xe4r\xa6\x06\xbf\x8bbf\xdd0\xfd\xe2\x98%\xb3\x10\x91"\x886\xd0\x94\xce\x8f\x96\xdb\xbd\xf1\xd2\x13\\\x838F@\x1eB\xb6\xa3\xc3H~nk:(T\xfa\x85\xba=\xca^\x9b\x9f\n\x15y+N\xd4\xe5\xacs\xf3\xa7W\x07p\xc0\xf7\x8c\x80c\rgJ\xde\xed1\xc5\xfe\x18\xe3\xa5\x99w&\xb8\xb4|\x11D\x92\xd9# \x89.7?\xd1[\x95\xbc\xcf\xcd\x90\x87\x97\xb2\xdc\xfc\xbea\xf2V\xd3\xab\x14*]\x9e\x84<9SGmA\xa2\x1f-C\xd8\xb7{\xa4v\xc4\x17I\xec\x7f\x0co\xf6l\xa1;R)\x9dU\xaa\xfb`\x86\xb1\xbb\xcc>Z\xcbY_\xb0\x9c\xa9\xa0Q\x0b\xf5\x16\xebzu,\xd7O\xae\xd5\xe9\xe6\xe7\xad\xe8t\xd6\xf4\xea\xa8PX\xaf'),
 bytearray(b'\x01\x02\x04\x08\x10 @\x80\x1d:t\xe8\xcd\x87\x13&L\x98-Z\xb4u\xea\xc9\x8f\x03\x06\x0c\x180`\xc0\x9d\'N\x9c%J\x945j\xd4\xb5w\xee\xc1\x9f#F\x8c\x05\n\x14(P\xa0]\xbai\xd2\xb9o\xde\xa1_\xbea\xc2\x99/^\xbce\xca\x89\x0f\x1e<x\xf0\xfd\xe7\xd3

In [95]:
prim = rs.find_prime_polys(c_exp=12, fast_primes=True, single=True)

In [96]:
rs.init_tables(c_exp=12, prim=prim)

[array('i', [0, 0, 1, 3825, 2, 3555, 3826, 1600, 3, 1330, 3556, 822, 3827, 1222, 1601, 3285, 4, 3015, 1331, 339, 3557, 3200, 823, 952, 3828, 2091, 1223, 1060, 1602, 552, 3286, 590, 5, 320, 3016, 2822, 1332, 394, 340, 282, 3558, 2666, 3201, 1821, 824, 790, 953, 852, 3829, 2422, 2092, 2745, 1224, 69, 1061, 2516, 1603, 682, 553, 3119, 3287, 566, 591, 2930, 6, 2660, 321, 531, 3017, 1644, 2823, 296, 1333, 1779, 395, 412, 341, 2849, 283, 3691, 3559, 2444, 2667, 2152, 3202, 2475, 1822, 1948, 825, 2246, 791, 207, 954, 2190, 853, 3894, 3830, 3112, 2423, 50, 2093, 2552, 2746, 807, 1225, 12, 70, 705, 1062, 642, 2517, 124, 1604, 582, 683, 3974, 554, 3190, 3120, 520, 3288, 1939, 567, 2396, 592, 1551, 2931, 2044, 7, 1774, 2661, 1325, 322, 21, 532, 1281, 3018, 1286, 1645, 1669, 2824, 2126, 297, 1006, 1334, 3985, 1780, 312, 396, 3704, 413, 4022, 342, 250, 2850, 1812, 284, 941, 3692, 2920, 3560, 624, 2445, 2842, 2668, 3875, 2153, 3099, 3203, 537, 2476, 2981, 1823, 1161, 1949, 2282, 826, 3949, 2247, 216

# Reference

1. https://en.wikipedia.org/wiki/Reed%E2%80%93Solomon_error_correction
https://pypi.org/project/reedsolo/#table-of-contents

2. https://www.geeksforgeeks.org/what-is-reed-solomon-code/

3. https://pypi.org/project/unireedsolomon/

4. https://studybuff.com/how-do-you-create-a-reed-solomon-code/

5. https://en.wikiversity.org/wiki/Reed%E2%80%93Solomon_codes_for_coders

6. https://books.google.fr/books?id=eQs2i-R9-oYC&lpg=PR11&ots=atCPQJm3OJ&dq=%22Algebraic%20codes%20for%20data%20transmission%22%2C%20Blahut%2C%20Richard%20E.%2C%202003%2C%20Cambridge%20university%20press.&lr&hl=fr&pg=PA193#v=onepage&q=%22Algebraic%20codes%20for%20data%20transmission%22,%20Blahut,%20Richard%20E.,%202003,%20Cambridge%20university%20press.&f=false

7. https://math.stackexchange.com/questions/187302/reed-solomon-code-calculation

8. https://stackoverflow.com/questions/32355713/how-to-detect-errors-for-reed-solomon-code

9. https://github.com/brownan/Reed-Solomon

10. https://www.youtube.com/watch?v=b3NxrZOu_CE

11. https://www.youtube.com/watch?v=GvjLPJFAUOw&list=PLkvhuSoxwjI_UudECvFYArvG0cLbFlzSr&index=21

12. https://en.wikiversity.org/wiki/Reed%E2%80%93Solomon_codes_for_coders
